In [186]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [187]:
from lib import gauss_newton, levenberg_marquardt
import numpy as np
from typing import Callable

## Example functions

In [188]:
xs = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])
ys = np.array([1, 2, 3, 4])


def f(x: np.ndarray, p: np.ndarray) -> np.ndarray:
    assert x.shape == (2,)
    assert p.shape == (3,)

    return p[0] * x[0] + p[1] * x[1] + p[2]


def df(x: np.ndarray, p: np.ndarray) -> np.ndarray:
    assert x.shape == (2,)
    assert p.shape == (3,)

    return np.array([x[0], x[1], 1])


def residue(f: Callable, xs: np.ndarray, ys: np.ndarray, p: np.ndarray) -> np.ndarray:
    assert xs.shape[1] == 2
    assert xs.shape[0] == ys.shape[0]
    assert p.shape == (3,)

    fs = np.array([f(x, p) for x in xs])

    return (fs - ys) ** 2


def residue_jacobian(
    f: Callable, df: Callable, ys: np.ndarray, xs: np.ndarray, p: np.ndarray
) -> np.ndarray:
    assert xs.shape[1] == 2
    assert xs.shape[0] == ys.shape[0]
    assert p.shape == (3,)

    fs = np.array([f(x, p) for x in xs])
    dfs = np.array([df(x, p) for x in xs])

    return np.array([2 * (fs[i] - ys[i]) * dfs[i] for i in range(len(xs))])


p0 = np.random.randn(3)
alpha = 0.01
max_iter = 1000

In [189]:
residue(f, xs, ys, p0)

array([ 4.40156389, 13.13662688, 26.53183824, 44.58719797])

In [190]:
residue_jacobian(f, df, ys, xs, p0)

array([[ -4.19598088,  -8.39196177,  -4.19598088],
       [-14.49779398, -21.74669096,  -7.24889699],
       [-30.90543927, -41.20725237, -10.30181309],
       [-53.41891678, -66.77364598, -13.3547292 ]])

In [191]:
def F(p: np.ndarray) -> np.ndarray:
    return residue(f, xs, ys, p)


def DF(p: np.ndarray) -> np.ndarray:
    return residue_jacobian(f, df, ys, xs, p)

## Gauss-Newton method

In [192]:
p, err = gauss_newton(F, DF, p0, max_iter, silent=True)
p, err

(array([ 1.5947076 , -0.59470833,  0.59470805]), 1.2210413896330348e-11)

In [193]:
np.array([f(xs[i], p) for i in range(len(xs))])

array([0.999999  , 1.99999827, 2.99999754, 3.99999682])

## Levenberg-Marquardt method

In [194]:
def lambda_fun(F: Callable, DF: Callable, p: np.ndarray, i: int) -> float:
    return 1e-3

In [195]:
p, err = levenberg_marquardt(lambda_fun, F, DF, p0, max_iter, silent=True)
p, err

(array([ 1.41556198, -0.41567348,  0.41597719]), 4.267427510309675e-08)

In [196]:
np.array([f(xs[i], p) for i in range(len(xs))])

array([1.00019221, 2.00008071, 2.99996922, 3.99985772])